In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


from model.feature_engineering import FeatureEngineering
from model.preprocessing import Preprocessing
from model.visualization import Visualization
from model.model import Model
from model.metrics import Metrics

from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /home/telson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/telson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
feature = FeatureEngineering()
preproc = Preprocessing()
visualization = Visualization()
model = Model()
metrics = Metrics()

In [3]:
df_genre_musics_train = pd.read_csv("train/train.csv")
count_vectorizer = CountVectorizer()
tf_idf_vectorizer = TfidfVectorizer()

In [4]:
count_vectorizer = model.vectorizer(count_vectorizer, df_genre_musics_train["lyric"])
text_vect = count_vectorizer.transform(df_genre_musics_train["lyric"])

df_count_vectorizer = pd.DataFrame(data = metrics.experiments_models(df_genre_musics_train["genre"], text_vect))
df_count_vectorizer.to_csv("results/count_vectorizer.csv")


BaggingClassifier: 0.7078125
RandomForestClassifier: 0.821484375


/home/telson/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/telson/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

LogisticRegression: 0.806640625
KNeighborsClassifier: 0.403125
DecisionTreeClassifier: 0.648828125


In [5]:
tf_idf = model.vectorizer(tf_idf_vectorizer, df_genre_musics_train["lyric"])
text_vectorizer = tf_idf.transform(df_genre_musics_train["lyric"])

df_tf_idf = pd.DataFrame(data= metrics.experiments_models(df_genre_musics_train['genre'], text_vectorizer))
df_tf_idf.to_csv("results/tf_idf.csv")


BaggingClassifier: 0.71015625
RandomForestClassifier: 0.8234375
LogisticRegression: 0.8359375
KNeighborsClassifier: 0.680078125
DecisionTreeClassifier: 0.656640625


In [6]:
## Modelo Escolhido - Tf-IDF -> LogisticRegression()
tf_idf = model.vectorizer(tf_idf_vectorizer, df_genre_musics_train["lyric"])
model.save_model(tf_idf, "vectorizer")

In [7]:
df_valid = pd.read_csv("valid/valid.csv")
valid_lyric = preproc.clean_lyric(df_valid.lyric)

In [8]:
train_lyric = tf_idf.transform(df_genre_musics_train["lyric"])
valid_lyric = tf_idf.transform(valid_lyric)

lr_model = metrics.benchmark(LogisticRegression(), train_lyric, df_genre_musics_train["genre"], valid_lyric, df_valid.genre)

________________________________________________________________________________
Training: 
LogisticRegression()
train time: 1.560s
test time:  0.004s
f1_score:   0.838
*************** Classification Report ***************
              precision    recall  f1-score   support

  bossa_nova       0.82      0.82      0.82       160
        funk       0.87      0.81      0.84       160
      gospel       0.91      0.88      0.89       160
   sertanejo       0.76      0.84      0.80       160

    accuracy                           0.84       640
   macro avg       0.84      0.84      0.84       640
weighted avg       0.84      0.84      0.84       640

Confusion Matrix



In [9]:
model.save_model(lr_model, "classifier")